In [ ]:
import numpy as np
import h5py
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error as mse
from glob import glob
from scipy import signal
from peakfinder import detect_peaks 
np.set_printoptions(suppress=True)

In [ ]:
occ1 = "../data/level_1p0a/20190505_184810_1p0a_UVIS_I.h5"
occ2 = "../data/level_1p0a/20190715_162335_1p0a_UVIS_E.h5"

In [ ]:
with h5py.File(occ1, "r") as f:
    occ1 = {}
    occ1["yMean"] = np.array(f["Science/YMean"])
    occ1["y"] = np.array(f["Science/Y"])
    occ1["x"] = np.array(f["Science/X"])
    
with h5py.File(occ2, "r") as f:
    occ2 = {}
    occ2["yMean"] = np.array(f["Science/YMean"])
    occ2["y"] = np.array(f["Science/Y"])
    occ2["x"] = np.array(f["Science/X"])

In [ ]:
def wl_clip(wl, wl_arr):
    return np.argmin(abs(wl_arr - wl))

def compress(arr, factor):
    """
    Compress an array by a given factor by taking averages of groups of points
    """
    if arr.shape[0] % factor:
        idx = arr.shape[0] % factor
        compressed = arr[:-idx].reshape(int(arr[:-idx].shape[0]/factor), int(factor)).sum(axis=1)/factor
        compressed = np.append(compressed, np.mean(arr[-idx:]))
    else:
        compressed = arr.reshape(int(arr.shape[0]/factor), int(factor)).sum(axis=1)/factor
    return compressed

def intersectCounter(f):
    """
    Counts the number of times f crosses x axis
    """
    sign = np.sign(f)
    count = np.array((np.roll(sign, -1) - sign)[:-1], dtype="bool").sum()
    return count

def fourier_extract(sounding, n_freq):
    """
    Fourier transform the sounding, extract largest amplitude (excluding that corresponding to zero frequency),
    filter the sounding such that n_freq frequencies remain, calculate mse between filtered signal and original signal.
    
    Inputs
    sounding     Spectrum to be analysed 
    n_freq       Number of remaining frequencies after being filtered via fourier transform
    
    Outputs
    error        Mean squared error between original and smoothed spectra
    amp          Largest amplitude in fourier transform (excluding zero frequency)
    period       Period corresponding with the largest amplitude (excluding zero frequency)
    """
    transformed = np.fft.fft(sounding)
    cleanTransformed = filter_n(transformed, n_freq)
    cleanSounding = np.fft.ifft(cleanTransformed)
    error = mse(sounding, abs(cleanSounding))
    idx = np.argmax(abs(transformed)[1:51]) + 1
    amp = abs(transformed[1:51]).max()
    period = 1/freq[idx]
    return np.array([error, amp, period])

In [ ]:
fig = plt.figure(figsize=(20,9))
ax1 = fig.add_subplot(1, 2, 1, projection='3d')
ax2 = fig.add_subplot(1, 2, 2, projection='3d')

for i in range(occ1["x"].shape[0]):
    ax1.plot(occ1["x"][i], occ1["y"][i], zs=i, zdir="y");
    
for i in range(occ2["x"].shape[0]):
    ax2.plot(occ2["x"][i], occ2["y"][i], zs=i, zdir="y");
    
ax1.set_title("Occultation 1")
ax1.set_xlabel('Wavelength (nm)')
ax1.set_ylabel('Sounding')
ax1.set_zlabel('Transmission');

ax1.set_title("Occultation 2")
ax2.set_xlabel('Wavelength (nm)')
ax2.set_ylabel('Sounding')
ax2.set_zlabel('Transmission');

In [ ]:
fig = plt.figure(figsize=(20,9))
ax1 = fig.add_subplot(1, 2, 1)
ax2 = fig.add_subplot(1, 2, 2)

for i in range(occ1["x"].shape[0]):
    ax1.plot(occ1["x"][i], occ1["y"][i]);

for i in range(occ2["x"].shape[0]):
    ax2.plot(occ2["x"][i], occ2["y"][i]);

In [ ]:
clip = wl_clip(300, occ1["x"][0])

wavelength = occ1["x"][0][clip:]
s1 = occ1["y"][90][clip:]
s2 = occ2["y"][40][clip:]
s3 = occ2["y"][127][clip:]

fig = plt.figure(figsize=(20,9))
ax = fig.add_subplot()

ax.plot(wavelength, s1)
ax.plot(wavelength, s2)
ax.plot(wavelength, s3)

ax.legend(["s1", "s2", "s3"]);

In [ ]:
plt.plot(s1)
plt.plot(np.poly1d(np.polyfit(wavelength, s1, 5))(wavelength))

In [ ]:
coeffs1 = np.polyfit(wavelength, s1, 3)
fit1 = np.poly1d(coeffs1)(wavelength)
res1 = np.abs(fit1 - s1)
coeffs2 = np.polyfit(wavelength, s2, 3)
fit2 = np.poly1d(coeffs2)(wavelength)
res2 = np.abs(fit2 - s2)
coeffs3 = np.polyfit(wavelength, s3, 3)
fit3 = np.poly1d(coeffs3)(wavelength)
res3 = np.abs(fit3 - s3)

plt.semilogy(res1)
plt.semilogy(res2)
plt.semilogy(res3)
plt.legend(["s1", "s2", "s3"]);

In [ ]:
plt.plot(fit1)
plt.plot(s1)
plt.plot(fit2)
plt.plot(s2)
plt.plot(fit3)
plt.plot(s3)

Objective is to find a quantifiable feature that distinguishes the signal s1 from the signals s2 and s3!

# Fourier

If there is one nan value in an array, the fourier transform will be all nan values. Linear interpolation is used to fill the np.nan values.
###### TODO
array could be truncated to remove nan values rather than linear interpolation, depends on where nan values could occur.

In [ ]:
mask = np.isfinite(s1)
s1 = np.interp(wavelength, wavelength[mask], s1[mask])
mask = np.isfinite(s2)
s2 = np.interp(wavelength, wavelength[mask], s2[mask])
mask = np.isfinite(s3)
s3 = np.interp(wavelength, wavelength[mask], s3[mask])

fs1 = np.fft.fft(s1)
fs2 = np.fft.fft(s2)
fs3 = np.fft.fft(s3)
freq = np.fft.fftfreq(wavelength.shape[0])

fig = plt.figure(figsize=(20,9))
ax = fig.add_subplot()

ax.semilogy(freq, abs(fs1))
ax.semilogy(freq, abs(fs2))
ax.semilogy(freq, abs(fs3));

#ax.set_ylim([0,2])
#ax.set_xlim([-0.1,0.1])
ax.legend(["fs1", "fs2", "fs3"])
ax.grid();

In [ ]:
freqidx = np.argmax(abs(fs2)[1:]) + 1
abs(freq[freqidx])

In [ ]:
1/freq[freqidx]

In [ ]:
abs(fs2[1:]).max()

In [ ]:
thresh = 0.25

fs1c = np.where(abs(fs1) > thresh, fs1, 0+0j)
fs2c = np.where(abs(fs2) > thresh, fs2, 0+0j)
fs3c = np.where(abs(fs3) > thresh, fs3, 0+0j)

s1c = np.fft.ifft(fs1c)
s2c = np.fft.ifft(fs2c)
s3c = np.fft.ifft(fs3c)

fig = plt.figure(figsize=(20,9))
ax = fig.add_subplot()

ax.plot(wavelength, abs(s1c))
ax.plot(wavelength, abs(s2c))
ax.plot(wavelength, abs(s3c));

ax.legend(["s1c", "s2c", "s3c"])
ax.grid();

All signals are superpositions of periodic signals with a maximum period of 128. The exact combination of frequencies varies per sounding. The soundings of interest should have a spike, which indicates a "main" waveform frequency that the original noisy data somewhat resembles.

The local maximum of the fourier transform (excluding the zero frequency) may be useful. Also the shape of the transform and its "spikiness".

###### Idea 1
Take fourier transform, filter until only n (small n) frequencies remain, reverse fourier transform and then measure MSE between result and original signal. Assumption here is that a filtered signal that can reasonably approximate s3 requires more frequencies than one that can approximate s1.
###### Idea 2
Investigate autocorrelation of interpolating function that approximates original signal.

In [ ]:
def filter_n(transform, n):
    """
    Takes a fourier transform of a signal as input and filters it until only n non-zero frequencies remain
    """
    return np.where(abs(transform) >= np.sort(abs(transform))[-2*n], transform, 0+0j)

In [ ]:
fs1c = filter_n(fs1, 2)
fs2c = filter_n(fs2, 2)
fs3c = filter_n(fs3, 2) #TODO filter freq

s1c = np.fft.ifft(fs1c)
s2c = np.fft.ifft(fs2c)
s3c = np.fft.ifft(fs3c)

fig = plt.figure(figsize=(20,9))
ax = fig.add_subplot()

ax.plot(wavelength, abs(s1c))
ax.plot(wavelength, abs(s2c))
ax.plot(wavelength, abs(s3c));

ax.legend(["s1c", "s2c", "s3c"])
ax.grid();

In [ ]:
print(mse(s1, abs(s1c)), mse(s2, abs(s2c)), mse(s3, abs(s3c)))

This can distinguish s1 and s2. The basic fourier transform can distinguish between s3 and the rest.

Find amplitude of dominant non-zero frequency, find mse between original and filtered signal, then cluster.

uniform shifts -> linear shifts -> non-linear shifts -> periodic

In [ ]:
fig = plt.figure(figsize=(20,3))
ax1 = fig.add_subplot(1,3,1)
ax2 = fig.add_subplot(1,3,2)
ax3 = fig.add_subplot(1,3,3)
ax1.imshow(signal.cwt(s1, signal.ricker, np.arange(1, 51)))
ax2.imshow(signal.cwt(s2, signal.ricker, np.arange(1, 51)))
ax3.imshow(signal.cwt(s3, signal.ricker, np.arange(1, 51)))

# Autocorrelation

In [ ]:
fig = plt.figure(figsize=(20,6))
ax1 = fig.add_subplot(1,3,1)
ax2 = fig.add_subplot(1,3,2)
ax3 = fig.add_subplot(1,3,3)

cor1 = np.array([])
cor2 = np.array([])
cor3 = np.array([])

for i in range(s1.shape[0]):
    cor1 = np.append(cor1, np.correlate(np.roll(s1, i), s1))
    
for i in range(s2.shape[0]):
    cor2 = np.append(cor2, np.correlate(np.roll(s2, i), s2))
    
for i in range(s3.shape[0]):
    cor3 = np.append(cor3, np.correlate(np.roll(s3, i), s3))
    
ax1.plot(cor1)
ax2.plot(cor2)
ax3.plot(cor3)

In [ ]:
fig = plt.figure(figsize=(20,6))
ax1 = fig.add_subplot(1,3,1)
ax2 = fig.add_subplot(1,3,2)
ax3 = fig.add_subplot(1,3,3)

ax1.plot(abs(np.correlate(s1c, s1c, mode="full")))
ax2.plot(abs(np.correlate(s2c, s3c, mode="full")))
ax3.plot(abs(np.correlate(s2c, s3c, mode="full")))

# Phase coherence
https://www.jstor.org/stable/pdf/3546310.pdf?refreqid=excelsior%3Aab23964ed57763344ab0eab4b0cd4e16

In [ ]:
plt.plot(s1)

In [ ]:
std = np.zeros((s1.shape[0]-1, s1.shape[0]-1))
for i in range(s1.shape[0]-1):
    for j in range(i+1):
        std[i,j] = s1[j::i+1].std()

In [ ]:
#std[step:start]
plt.plot(std[:,0])
plt.ylabel("std")
plt.xlabel("step");

In [ ]:
detect_peaks(std[:,0], mpd=40, show=True, xdata=np.arange(100))

In [ ]:
std = np.zeros((s2.shape[0], s2.shape[0]))
for i in range(1, s2.shape[0]):
    for j in range(i):
        std[i-1,j] = s2[j::i].std()

In [ ]:
plt.plot(std[:,0])
plt.ylabel("std")
plt.xlabel("step");

In [ ]:
std = np.zeros((s3.shape[0], s3.shape[0]))
for i in range(1, s3.shape[0]):
    for j in range(i):
        std[i-1,j] = s3[j::i].std()

In [ ]:
plt.plot(std[:,0])
plt.ylabel("std")
plt.xlabel("step");

# Polyfit

In [ ]:
res = []
for n in range(10):
    coeffs = np.polyfit(wavelength, s1, n)
    fit = np.poly1d(coeffs)(wavelength)
    res.append(np.abs(fit - s1).sum())
    

coeffs = np.polyfit(wavelength, s1, 4);
fit = np.poly1d(coeffs)(wavelength);
plt.plot(fit)
plt.plot(s1)

# Moving average

In [ ]:
plt.plot(s1)

In [ ]:
plt.plot(compress(s1, 4))

In [ ]:
plt.plot(np.gradient(compress(s1, 32)))

In [ ]:
intersectCounter(np.gradient(compress(s1, 32)))

# Peak finder

In [ ]:
detect_peaks(s1, mph=s1.mean(), mpd=20, show=True, xdata=wavelength)

In [ ]:
detect_peaks(s1, mph=s1.mean(), mpd=20, show=True, xdata=wavelength, valley=True)

In [ ]:
detect_peaks(s2, mph=s1.mean(), mpd=20, show=True, xdata=wavelength)

In [ ]:
detect_peaks(s3, mph=s1.mean(), mpd=20, show=True, xdata=wavelength)